In [1]:
import pandas as pd

### Load dataset

In [2]:
fao_domestic_supply = pd.read_csv('data/FAO_Domestic_Supply.csv')

In [3]:
fao_domestic_supply.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,New Food Balances,2,Afghanistan,5301,Domestic supply quantity,2511,Wheat and products,2014,2014,1000 tonnes,6546.0,S,Standardized data
1,FBS,New Food Balances,2,Afghanistan,5301,Domestic supply quantity,2511,Wheat and products,2015,2015,1000 tonnes,6545.0,S,Standardized data
2,FBS,New Food Balances,2,Afghanistan,5301,Domestic supply quantity,2511,Wheat and products,2016,2016,1000 tonnes,7287.0,S,Standardized data
3,FBS,New Food Balances,2,Afghanistan,5301,Domestic supply quantity,2511,Wheat and products,2017,2017,1000 tonnes,7326.0,S,Standardized data
4,FBS,New Food Balances,2,Afghanistan,5301,Domestic supply quantity,2511,Wheat and products,2018,2018,1000 tonnes,6995.0,S,Standardized data


### Group food items by food groups

In [4]:
gbl = ['Wheat and products', 'Rice and products', 'Barley and products', 
       'Maize and products', 'Rye and products', 'Oats', 'Millet and products',
       'Sorghum and products', 'Cereals, Other','Cassava and products', 
       'Beans', 'Pulses, Other and products','Nuts and products', 'Soyabeans',
       'Groundnuts (Shelled Eq)', 'Sunflower seed',
       'Rape and Mustardseed', 'Cottonseed', 'Peas', 'Sesame seed','Palm kernels','Oilcrops, Other',
      'Cocoa Beans and products',
      ]
dairy = ['Butter, Ghee', 'Cream','Milk - Excluding Butter',]
fruits = ['Coconuts - Incl Copra','Olives (including preserved)', 'Tomatoes and products'
         'Oranges, Mandarines',
       'Lemons, Limes and products', 'Grapefruit and products',
       'Citrus, Other', 'Bananas', 'Plantains', 'Apples and products',
       'Pineapples and products', 'Dates', 'Grapes and products (excl wine)','Fruits, Other',]
meat = ['Bovine Meat',
       'Mutton & Goat Meat', 'Pigmeat', 'Poultry Meat', 'Meat, Other',
        'Offals, Edible','Fats, Animals, Raw', 'Eggs','Freshwater Fish',
       'Demersal Fish',
       'Pelagic Fish', 'Marine Fish, Other', 'Crustaceans']
vegetables = ['Cassava and products', 'Potatoes and products', 
              'Sweet potatoes', 'Yams',
             'Roots, Other', 'Onions',
       'Vegetables, Other','Pepper', 'Pimento', 'Cloves',
       'Spices, Other', ]
confections = ['Sugar cane', 'Sugar beet',
       'Sugar non-centrifugal', 'Sugar (Raw Equivalent)',
              'Sweeteners, Other', 'Honey','Soyabean Oil', 'Groundnut Oil',
       'Sunflowerseed Oil', 'Rape and Mustard Oil', 'Cottonseed Oil',
       'Palmkernel Oil', 'Palm Oil', 'Coconut Oil', 'Sesameseed Oil',
       'Olive Oil', 'Ricebran Oil', 'Maize Germ Oil',
       'Oilcrops Oil, Other','Fish, Body Oil', 'Fish, Liver Oil',]
caffeine = ['Coffee and products','Tea (including mate)']
alcohol = ['Wine', 'Beer', 'Beverages, Fermented',
       'Beverages, Alcoholic', 'Alcohol, Non-Food']

In [5]:
fao_domestic_supply = fao_domestic_supply.rename(columns={'Area': 'Country Name'})

### Filter rows by food groups

In [6]:
gbl_df = fao_domestic_supply.loc[fao_domestic_supply['Item'].isin(gbl)]
dairy_df = fao_domestic_supply.loc[fao_domestic_supply['Item'].isin(dairy)]
fruits_df= fao_domestic_supply.loc[fao_domestic_supply['Item'].isin(fruits)]
meat_df = fao_domestic_supply.loc[fao_domestic_supply['Item'].isin(meat)]
vegetables_df = fao_domestic_supply.loc[fao_domestic_supply['Item'].isin(vegetables)]
confections_df =fao_domestic_supply.loc[fao_domestic_supply['Item'].isin(confections)]
caffeine_df = fao_domestic_supply.loc[fao_domestic_supply['Item'].isin(caffeine)]
alcohol_df = fao_domestic_supply.loc[fao_domestic_supply['Item'].isin(alcohol)]

### Sum food group values

In [7]:
gbl_consumption = gbl_df.groupby(['Country Name', 'Year']).sum().reset_index()[['Country Name', 'Year', 'Value']].rename(columns={'Value': 'GBL Consumption'})

In [8]:
def group_by_fg(df, name):
    """
        Group by Country and Year for each food consumption dataframe and sum the values.
    """
    return df.groupby(['Country Name', 'Year']).sum().reset_index()[['Country Name', 'Year', 'Value']].rename(columns={'Value': name})

### Create the food consumption datasets

In [9]:
dairy_consumption  = group_by_fg(dairy_df, 'Dairy Consumption')
fruits_consumption = group_by_fg(fruits_df, 'Fruits Consumption')
meat_consumption = group_by_fg(meat_df, 'Meat Consumption')
vegetables_consumption = group_by_fg(vegetables_df, 'Vegetables Consumption')
confections_consumption = group_by_fg(confections_df, 'Confections Consumption')
caffeine_consumption = group_by_fg(caffeine_df, 'Caffeine Consumption')
alcohol_consumption = group_by_fg(alcohol_df, 'Alcohol Consumption')

### Merge the food group consumptions into a single CSV

In [10]:
dataframes = [gbl_consumption,
dairy_consumption,
fruits_consumption,
meat_consumption,
vegetables_consumption,
confections_consumption,
caffeine_consumption,
alcohol_consumption]

In [11]:
from functools import reduce
joined = reduce(lambda left,right: pd.merge(left,right,on=['Country Name', 'Year'],
                                            how='inner'), dataframes)

#### Result

In [12]:
joined.head()

,Country Name,Year,GBL Consumption,Dairy Consumption,Fruits Consumption,Meat Consumption,Vegetables Consumption,Confections Consumption,Caffeine Consumption,Alcohol Consumption
0,Afghanistan,2014,8371.0,1897.0,1072.0,562.36,1873.0,684.0,110.0,1.0
1,Afghanistan,2015,8105.0,2263.0,1203.0,551.20,1871.0,676.0,82.0,0.0
2,Afghanistan,2016,8773.0,2273.0,1039.0,531.05,3154.0,628.0,51.0,30.0
3,Afghanistan,2017,8656.0,2221.0,1207.0,520.00,2405.0,712.0,79.0,0.0
4,Afghanistan,2018,8219.0,2220.0,831.0,509.00,1886.0,763.0,81.0,0.0


### Save File

In [13]:
joined.to_csv('data/FAO_Consumption.csv')